Estudiaremos en el presente Notebook las cotizaciones y las distintas carácterísticas que componen los ETFs de BlackRock, conocidos como iShares.
Todos ellos serán los cotizados en **USD-$**, para poder realizar comparativas y agrupaciones de forma más sencilla.

Debido a esta condición, añadida a un mínimo de 20 años desde su salida al mercado para tener un número considerable de registros, nos encontramos ante **87 ETFs** distintos para analizar.


# Data Project

## Dataset

Nos quedará un Dataset compuesto por el histórico de la capitalización de cada ETF de iShare, que añadiremos a nuestro entorno de desarrollo en la carpeta de *pre-processed*, donde incluiremos todos los csv.

Estos csv han sido generados despúes de un primer paso de darle calidad a nuestros datos a partir de un script de Python que transforma los datos de la carpeta *raw* a los csv de *pre-processed*, preparándolos para su uso en KDB+.

En nuestro mismo directorio, recibiremos otro csv, donde estará la información a cargar para la tabla Resume, también generado por Python.


  > Tabla **Historical**: Registra el histórico de cotización de los ETFs. 
  >  - ***date***: La fecha del registro del histórico.    
  > - ***nav_value***: El valor de la participación en esa fecha.    
  > - ***shares***: El nº de participaciones en el mercado del ETF en esa fecha.  
  > - ***ticker***: El ticker identificativo del ETF.

  > Tabla **Resume**: Se trata de la información general del los ETFs de iShare. 
  >  - ***ticker***: El identificativo de cada ETF de iShares.    
  > - ***inception_date***: La fecha a la que sale al mercado el ETF.    
  > - ***net_assets***: El total de capitalización del ETF en el momento de sacar los datos.  
  > - ***equity_class***: Muestra el tipo de activo que compone el ETF. *Large/Mid Cap - All Cap - Large Cap - Mid Cap - Small Cap*
  > - ***region***: Muestra la región de dónde son las empresas. *Asia Pacific - North America  - Europe - Latin America - Global - Middle East and Africa*
  > - ***market***: Muestra el tipo de mercado que tiene las empresas. *Developed - Emerging*

1. Crearemos en un primer momento las tablas vacías.
2. Importaremos desde los csv la información y la insertaremos en nuestras tablas.

In [ ]:
 historical:([]date:`date$();nav_value:`float$();shares:`long$();ticker:`$())
 resume_etf:([ticker:`$()]inception_date:`date$();net_assets:`float$();equity_class:`$();region:`$();market:`$())
 @[`historical;`ticker;`g#];
 meta historical

## Data Processing - Filter and Quality
Nuestros datos están disponibles en la carpeta *Data*, pero se presentan en un formato de Excel y con información poco valiosa y muy dispersa para el estudio. Por tanto consideramos que se encuentran en un proceso raw en nuestro proyecto de Ingeniería de Datos.

Por ello contamos con 2 scripts a realizar, donde filtraremos, renombraremos y daremos calidad a los datos, para posteriormente poder estudiarlos de mejor manera.

Pero antes de ello realizaremos la instalación de las librerías correspondientes, actualizadas en el *requirements.txt*:

In [ ]:
system "pip install -r requirements.txt"

In [ ]:
system "python3 Data/script_etf_a_csv.py Data/RawExcels/ Data/iShares-UnitedStates2.xlsx"
system "python3 Data/script_resume_etf.py Data/iShares-UnitedStates2.xlsx"

## Data Ingest

Procedemos a importar los archivos de los ETFs que se encuentran en la ruta : *Data/PreProcessed/etf/* y los cargaremos  para poder operar con ellos.

In [2]:
csv_files: system "ls Data/PreProcessed/etf/*.csv";

Insertamos en nuestra tabla **'historical'**  usando un iterador y las características de programación funcional y su compilación de derecha a izquierda que nos aporta el lenguaje.

Importamos los datos del csv, actualizamos la columna ticker con el nombre que queremos y realizamos un upsert sobre la tabla.

In [3]:
//Ingesta de HISTORICAL
{ ticker_name: ssr[ssr[; "Data/PreProcessed/etf/"; ""] x; ".csv"; ""]; 
    `historical upsert update ticker:`$ticker_name from ("DFJ"; enlist csv) 0: `$x;
 } each csv_files;
//Ingesta de RESUME
`resume_etf insert ("SDFSSS"; enlist csv) 0: `$"Data/PreProcessed/resume.csv";

In [ ]:
show historical

In [ ]:
select count i from historical where ticker=`EWJ

In [ ]:
count historical

# New Data and Metadata

Procederemos a crear columnas en nuestras tablas con intención de recopilar información sobre el histórico y poder analizar posteriormente con mayor información y datos.

## Daily Returns:
 Añadiremos la diferencia de precios diaria en la tabla de *Historical* para poder calcular la volatilidad total.

 Por ello, lo primero que haremos será ordenar por ticker y fecha paraque se calcule de manera correcta los daily returns.

In [4]:
historical: `ticker`date xasc historical;
historical: update daily_retns: (nav_value-prev nav_value)%prev nav_value by ticker from historical
show historical 

date       nav_value shares  ticker daily_retns  
-------------------------------------------------
2003.11.03 50.51092  150000  DVY                 
2003.11.04 50.35961  150000  DVY    -0.00299557  
2003.11.05 50.23249  150000  DVY    -0.002524344 
2003.11.06 50.47377  150000  DVY    0.004803226  
2003.11.07 50.58691  150000  DVY    0.002241719  
2003.11.10 50.18715  350000  DVY    -0.007902518 
2003.11.11 50.08851  350000  DVY    -0.001965463 
2003.11.12 50.61071  550000  DVY    0.01042557   
2003.11.13 50.74544  550000  DVY    0.002662164  
2003.11.14 50.54792  900000  DVY    -0.003892507 
2003.11.17 50.3196   900000  DVY    -0.004516823 
2003.11.18 49.84921  900000  DVY    -0.009348047 
2003.11.19 50.0617   1200000 DVY    0.004262715  
2003.11.20 49.78395  1200000 DVY    -0.005548253 
2003.11.21 49.89686  1500000 DVY    0.00226796   
2003.11.24 50.63929  1500000 DVY    0.01487943   
2003.11.25 51.0226   1800000 DVY    0.00756932   
2003.11.26 51.21912  1800000 DVY    0.003851685  


## Volatility
Añadimos la volatilidad total por ticker a la tabla *Resume*.

Tomaremos la fórmula de la Volatilidad cómo: 
> Volatility = σ<sub>period</sub> * √T<sub>period</sub>

En mercados financieros los años se tratan de 252 días de mercados abiertos, por tanto:

 - Volatilidad Anual: 252 días
 - Volatilidad 6 meses: 126 días
 - Volatilidad 3 meses: 63 días

### Historical Volatility - 1 Year

In [5]:
year_vol: select vol1y: dev daily_retns * sqrt[252] by ticker from historical where date > max date - 251

### Historical Volatility - 6 Months

In [6]:
sixm_vol: select  vol6m: var daily_retns * sqrt 126 by ticker from historical where date > max date - 125

### Historical Volatility - 3 Months

In [7]:
threem_vol: select vol3m: var daily_retns * sqrt 63 by ticker from historical where date > max date - 62

Finalmente añadimos a la table *resume_etf* por cada etf de las 3 volatilidades calculadas.

In [8]:
meta resume_etf: ((resume_etf lj `ticker xkey year_vol) lj `ticker xkey sixm_vol) lj `ticker xkey threem_vol

c             | t f a
--------------| -----
ticker        | s    
inception_date| d    
net_assets    | f    
equity_class  | s    
region        | s    
market        | s    
vol1y         | f    
vol6m         | f    
vol3m         | f    


## Moving Averages

Calcularemos las medias móviles de ciertos periodos:
 - 200 días
 - 200 meses


### SMA 200 Days

In [9]:
historical: `ticker`date xdesc historical;
historical: update d200sma:mavg[200;nav_value] by ticker from historical

### SMA 200 Months

Usamos la ordenación descendente + first en el fby para coger el primer dato del mes (el último al estar ordenado descendentemente).

In [10]:
historical: `ticker`date xdesc historical
show historical: update m200sma:mavg[200;nav_value] by ticker from historical where date = (first;date)fby `month$date

date       nav_value shares   ticker daily_retns  d200sma  m200sma 
-------------------------------------------------------------------
2024.08.30 231.1647  66150000 SOXX   0.02627348   231.1647 231.1647
2024.08.29 225.2466  66000000 SOXX   -0.00292115  228.2057         
2024.08.28 225.9066  66550000 SOXX   -0.01796347  227.4393         
2024.08.27 230.0389  66350000 SOXX   0.01140704   228.0892         
2024.08.26 227.4444  66300000 SOXX   -0.02581257  227.9602         
2024.08.23 233.4709  65650000 SOXX   0.02704338   228.8787         
2024.08.22 227.3233  65250000 SOXX   -0.03448592  228.6565         
2024.08.21 235.4427  65200000 SOXX   0.01487777   229.5047         
2024.08.20 231.9912  64900000 SOXX   -0.01361237  229.781          
2024.08.19 235.1928  65100000 SOXX   0.01742708   230.3222         
2024.08.16 231.1642  64400000 SOXX   -0.00282512  230.3987         
2024.08.15 231.8192  64100000 SOXX   0.04858268   230.5171         
2024.08.14 221.0786  63200000 SOXX   -0.00298517

In [11]:
//Vemos los días del mes que son seleccionados para calcular la sma mensual
select from historical where ticker = `EWA, m200sma <> 0N

date       nav_value shares   ticker daily_retns   d200sma  m200sma 
--------------------------------------------------------------------
2024.08.30 25.88643  55600000 EWA    -0.007072695  25.88643 25.88643
2024.07.31 25.04421  59200000 EWA    0.01550802    24.89389 25.46532
2024.06.28 24.45022  65000000 EWA    7.239724e-05  24.90141 25.12695
2024.05.31 24.67653  67000000 EWA    0.01018097    24.78494 25.01435
2024.04.30 23.35809  71600000 EWA    -0.02398628   24.71629 24.6831 
2024.03.31 24.64454  75800000 EWA    0             24.58409 24.67667
2024.02.29 23.96077  81400000 EWA    0.009388952   24.54468 24.5744 
2024.01.31 23.63226  84200000 EWA    -0.00698352   24.4245  24.45663
2023.11.30 22.47515  90200000 EWA    0.003628176   24.23681 24.23647
2023.10.31 20.70762  90200000 EWA    -0.00382065   23.9085  23.88358
2023.08.31 22.23646  93200000 EWA    -0.001556813  23.25353 23.73384
2023.07.31 23.37513  92600000 EWA    0.01098248    22.95047 23.70395
2023.06.30 22.56923  95600000 EWA 

## Period Performance
Calcularemos los rendimientos del etf en los siguientes periodos, en un valor porcentual(%):
- 5 años
- 1 año
- 6 meses

### Performance - 5 Years

In [12]:
perf_5y: select perf_5y: 100*(first nav_value - last nav_value)%last nav_value by ticker 
    from select from historical where (date = max date) or (date = max date-5*365)

In [13]:
performance: { [initial_date; period]
 period2: select from historical where (date = initial_date) or (date = initial_date - period*365); 
 select performance:100*(first nav_value - last nav_value)%last nav_value by ticker from period2
 }

In [14]:
performance[2007.10.10;6]

ticker| performance
------| -----------
DVY   | 0          
EEM   | 0          
EFA   | 117.1842   
EPP   | 0          
EWA   | 286.3624   
EWC   | 229.6995   
EWD   | 218.5423   
EWG   | 157.0636   
EWH   | 166.6597   
EWI   | 125.2692   
EWJ   | 69.5595    
EWK   | 152.5342   
EWL   | 125.5268   
EWM   | 169.0554   
EWN   | 94.56887   
EWO   | 455.4078   
EWP   | 227.1234   
EWQ   | 119.3636   
EWS   | 245.7083   
EWT   | 134.962    
..


### Performance - 3 Years

In [15]:
perf_3y: select perf_3y: 100*(first nav_value - last nav_value)%last nav_value by ticker 
    from select from historical where (date = max date) or (date = max date- 3*365 - 1)

### Performance - 1 Year

In [16]:
perf_1y: select perf_1y: 100*(first nav_value - last nav_value)%last nav_value by ticker 
    from select from historical where (date = max date) or (date = max date- 1*365)

Al final añadimos a la table *resume_etf* los datos calculados por cada etf de los 3 rendimientos.

In [17]:
meta resume_etf: ((resume_etf lj `ticker xkey perf_5y) lj `ticker xkey perf_3y) lj `ticker xkey perf_1y

c             | t f a
--------------| -----
ticker        | s    
inception_date| d    
net_assets    | f    
equity_class  | s    
region        | s    
market        | s    
vol1y         | f    
vol6m         | f    
vol3m         | f    
perf_5y       | f    
perf_3y       | f    
perf_1y       | f    


## Drawdowns

Calcularemos:
- 5 años
- 3 años
- 1 año

In [18]:
drawdown: {[table; period]
    table: `ticker`date xasc table;
    max_values: select date, ticker, max_value: nav_value
        from table where date >= max date - period * 365, 
        nav_value = (max; nav_value) fby ticker;
    w: flip max_values[`date],'max table[`date];
    min_values: select date, ticker, min_value: nav_value from table;
    filter_data: wj [w; `ticker`date; max_values; (min_values; (min; `min_value))];
    select ticker,drawdown: 100 * (max_value - min_value) % min_value from filter_data
 }

### Drawdown - 5 Years

In [19]:
drw_5y: delete drawdown from `ticker xcol update drw_5y: drawdown from drawdown[historical;5]

### Drawdown - 3 Years

In [20]:
drw_3y: delete drawdown from `ticker xcol update drw_3y: drawdown from drawdown[historical;3]

### Drawdown - 1 Year

In [21]:
drw_1y: delete drawdown from `ticker xcol update drw_1y: drawdown from drawdown[historical;1]

Finalmente añadimos a la table *resume_etf* los datos calculados por cada etf de los 3 drawdowns.

In [22]:
meta resume_etf: ((resume_etf lj `ticker xkey drw_5y)lj `ticker xkey drw_3y) lj `ticker xkey drw_1y

c             | t f a
--------------| -----
ticker        | s    
inception_date| d    
net_assets    | f    
equity_class  | s    
region        | s    
market        | s    
vol1y         | f    
vol6m         | f    
vol3m         | f    
perf_5y       | f    
perf_3y       | f    
perf_1y       | f    
drw_5y        | f    
drw_3y        | f    
drw_1y        | f    


## Inflows/Outflows

Calculamos las variaciones de valor total de los etf, con la ampliación/salida de dinero en el mercado del etf por la variación mensual de las participaciones en circulación.

In [23]:
historical: `ticker`date xasc historical

In [24]:
historical: update in_out_flow_per: 100 * (shares - prev shares) % prev shares,
    in_out_flow_val: nav_value * shares - prev shares
    by ticker from historical where date = (last;date)fby `month$date

### Maximum and Minimum

Guardaremos las fechas de mlos máximos y mínimos historicos de cotización en la tabla de resume_etf, para poder facilitar y agilizar acceder a ese valor.

In [25]:
resume_etf : resume_etf lj `ticker xkey (select ticker,max_day:date from historical where nav_value=(max; nav_value) fby ticker) 
resume_etf : resume_etf lj `ticker xkey (select ticker,min_day:date from historical where nav_value=(min; nav_value) fby ticker)

# Save to Data Warehouse

Una vez hemos generado toda la información y las tablas están listas, estas las guardaremos en disco.

Esto nos dará esa modularidad y persistencia de los datos para otros procesos futuros, pudiendo acudir a unos  datos ya elaborados y normalizados sin tener que acudir a los datos en bruto del DataLake. 

Guardaremos nuestras tablas en la carpeta de tablas del Data Warehouse.

### Guardado como Binarios

In [35]:

\ts `:Data/DataWarehouse/Tables/historical set historical

\ts `:Data/DataWarehouse/Tables/resume_etf set resume_etf


39 4194592


0 4194592


### Guardado Particionado

In [50]:
\ Tenemos que particionar y crear un enum diferenciado para todos los elementos de ticker
\ts `:Data/DataWarehouse/Tables/historicalSplayed/historical/ set .Q.en[`:Data/DataWarehouse/Tables/historicalSplayed/] historical;


24 12583696


In [51]:
\ts `:Data/DataWarehouse/Tables/resume_etfSplayed/resume_etf set .Q.en[`:Data/DataWarehouse/Tables/resume_etfSplayed/] resume_etf;

0 4199824


# Data Analysis: "The Ideal Conditions for ETFs to invest in!"

La idea es analizar los datos recopilados y poder acotar bajo que *parámetros/características* nuestros etf se comportan de cierto modo. 

El objetivo: **Poder hacer prediciones y modelos de las cotizaciones con mayores garantías.**

Nos encontramos ahora con unas tablas enriquecidas con mucha información de la siguiente manera:

> Tabla **Historical**: A parte de las ya existentes **date**, **nav_value**, **shares**, **ticker** se añade:
> - ***Daily Returns***: El aumento/pérdida de valor de manera % de un ETF respecto al dato previo.
> - ***SMA 200 Days***: El cálculo de la medía móvil de los últimos 200 días por cada ETF.
> - ***SMA 200 Months***: El cálculo de la medía móvil de los últimos 200 meses(último dato del mes disponible) por cada ETF.
> - ***Inflows/Outflows %***: Las entradas/salidas de dinero a nuestro ETF de manera mensual, con la ampliación/reducción de participaciones(shares) disponibles en el mercado.
> - ***Inflows/Outflows Value***: El valor en $ de la variación de participaciones en el mercado mensual.

> Tabla **Resume**: A parte de las ya existentes **ticker**, **inception_date**, **net_assets**, **equity_class**, **region**, **market** se añade:
> - ***Volatility: 1y, 6m, 3m***: La volatilidad de los precios del ETF en esos periodos.
> - ***Performance: 5y, 3y, 1y***: La variación del valor del ETF en esos periodos.
> - ***Drawdowns: 5y, 3y, 1y***; La caída máxima del precio de cotización en esos periodos.

# Extras + info futuro


In [ ]:
count select ticker from resume_etf where market = `$"Developed"
select from historical where ticker = `EWA, m200sma <> 0N
count select ticker from resume_etf where market = `Emerging

In [ ]:
//Filtrado  mezclando las dos tablas
select from historical where ticker in exec ticker from resume_etf where market = `$"Emerging"
//Lo mismo
show a: select ticker from resume_etf where market = `$"Developed"
select from historical where ticker in a[`ticker]

In [ ]:
select distinct equity_class from resume_etf
select distinct region from resume_etf

In [ ]:
//maximo aumento
select from historical where daily_retns=(max; daily_retns) fby ticker